In [1]:
# Install required packages
%pip install openai python-dotenv


  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
Using cached python_dotenv-1.1.1-py3-none-any.whl (20 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
#!/usr/bin/env python3
"""
OpenAI Assistant Testing Script - TS Knee Portfolio
Following the exact patterns from assistant.txt
"""

import os
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI
from typing_extensions import override

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY'),
    default_headers={"OpenAI-Beta": "assistants=v2"}
)

In [6]:

def create_assistant_and_vector_store():
    """Create assistant and vector store following assistant.txt pattern"""
    print("🤖 Creating assistant...")
    
    # Create assistant first (like assistant.txt)
    assistant = client.beta.assistants.create(
        name="TS Knee Portfolio Assistant",
        instructions="You are an expert medical assistant specializing in TS Knee systems. Use your knowledge base to answer questions about surgical techniques and protocols.",
        model="gpt-4o",
        tools=[{"type": "file_search"}],
    )
    print(f"✅ Assistant created: {assistant.id}")
    
    # Create vector store (like assistant.txt)
    vector_store = client.beta.vector_stores.create(name="TS Knee Documents")
    print(f"✅ Vector store created: {vector_store.id}")
    
    # Ready the files for upload to OpenAI (exactly like assistant.txt)
    file_paths = ["data/ts_knee/TS_Knee_triathlon-TS-Brochure.pdf"]
    
    # Check if file exists
    if not Path(file_paths[0]).exists():
        print(f"❌ File not found: {file_paths[0]}")
        return None, None
    
    file_streams = [open(path, "rb") for path in file_paths]
    
    # Use the upload and poll SDK helper (exactly like assistant.txt)
    file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
        vector_store_id=vector_store.id, files=file_streams
    )
    
    # Print status (like assistant.txt)
    print(f"✅ File batch status: {file_batch.status}")
    print(f"✅ File counts: {file_batch.file_counts}")
    
    # Close file streams
    for stream in file_streams:
        stream.close()
    
    # Update assistant with vector store (exactly like assistant.txt)
    assistant = client.beta.assistants.update(
        assistant_id=assistant.id,
        tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
    )
    print(f"✅ Assistant updated with vector store")
    
    return assistant, vector_store

def test_basic_question(assistant):
    """Test basic functionality without streaming"""
    print("\n" + "="*60)
    print("TESTING BASIC QUESTION")
    print("="*60)
    
    # Create a thread and attach message (like assistant.txt pattern)
    thread = client.beta.threads.create(
        messages=[
            {
                "role": "user",
                "content": "What is the Triathlon TS Knee system? Provide an overview.",
            }
        ]
    )
    
    # Run the assistant
    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )
    
    if run.status == 'completed':
        # Get messages
        messages = client.beta.threads.messages.list(
            thread_id=thread.id
        )
        
        # Process the latest assistant message (like assistant.txt)
        for message in messages.data:
            if message.role == 'assistant':
                message_content = message.content[0].text
                annotations = message_content.annotations
                citations = []
                
                # Process citations exactly like assistant.txt
                for index, annotation in enumerate(annotations):
                    message_content.value = message_content.value.replace(
                        annotation.text, f"[{index}]"
                    )
                    if file_citation := getattr(annotation, "file_citation", None):
                        cited_file = client.files.retrieve(file_citation.file_id)
                        citations.append(f"[{index}] {cited_file.filename}")
                
                print("RESPONSE:")
                print(message_content.value)
                print("\nCITATIONS:")
                print("\n".join(citations))
                break
    else:
        print(f"❌ Run failed with status: {run.status}")

class EventHandler():
    """Event handler exactly like assistant.txt"""
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        # Print citations exactly like assistant.txt
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("\n".join(citations))

In [7]:

def test_streaming_question(assistant):
    """Test streaming functionality exactly like assistant.txt"""
    print("\n" + "="*60)
    print("TESTING STREAMING RESPONSE")
    print("="*60)
    
    # Create thread with message (like assistant.txt)
    thread = client.beta.threads.create(
        messages=[
            {
                "role": "user",
                "content": "What are the key features and benefits of the TS Knee system? Include specific technical details.",
            }
        ]
    )
    
    # Use stream SDK helper exactly like assistant.txt
    with client.beta.threads.runs.stream(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="Please provide detailed information based on the TS Knee documentation.",
        event_handler=EventHandler(),
    ) as stream:
        stream.until_done()

def test_with_notes(assistant):
    """Test with additional notes context"""
    print("\n" + "="*60)
    print("TESTING WITH NOTES INTEGRATION")
    print("="*60)
    
    # Sample notes like in HHB system
    notes_context = """
ADDITIONAL NOTES FOR REFERENCE:

NOTE 1 - TS KNEE:
TITLE: Important Surgical Consideration
CONTENT: When using TS Knee system, always ensure proper alignment with femoral component. Double-check tibial rotation before final fixation.
---

NOTE 2 - GENERAL (SHARED):
TITLE: Patient Safety Protocol  
CONTENT: For all knee procedures, verify patient positioning and confirm surgical site marking before beginning.
---

USER MESSAGE: What should I consider during TS Knee surgery regarding alignment and safety?
"""
    
    # Create thread with notes context
    thread = client.beta.threads.create(
        messages=[
            {
                "role": "user",
                "content": notes_context,
            }
        ]
    )
    
    # Stream response with EventHandler
    with client.beta.threads.runs.stream(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="Please address the user question using both the uploaded documents and the provided notes.",
        event_handler=EventHandler(),
    ) as stream:
        stream.until_done()

In [10]:

def main():
    """Main function to run all tests"""
    print("🚀 Starting OpenAI Assistant Test")
    print(f"API Key present: {'Yes' if os.getenv('OPENAI_API_KEY') else 'No'}")
    
    if not os.getenv('OPENAI_API_KEY'):
        print("❌ Please set OPENAI_API_KEY environment variable")
        return
    
    # Create assistant and vector store
    assistant, vector_store = create_assistant_and_vector_store()
    
    if not assistant or not vector_store:
        print("❌ Failed to create assistant or vector store")
        return
    
    # Run tests
    test_basic_question(assistant)
    test_streaming_question(assistant)
    test_with_notes(assistant)
    
    # Print environment variables for production
    print("\n" + "="*60)
    print("PRODUCTION ENVIRONMENT VARIABLES")
    print("="*60)
    print(f"TS_KNEE_ASSISTANT_ID={assistant.id}")
    print(f"TS_KNEE_VECTOR_STORE_ID={vector_store.id}")
    
    print("\n✅ All tests completed!")

if __name__ == "__main__":
    main() 

🚀 Starting OpenAI Assistant Test
API Key present: Yes
🤖 Creating assistant...


BadRequestError: Error code: 400 - {'error': {'message': "The v1 Assistants API has been deprecated. Please try again by setting the header 'OpenAI-Beta: assistants=v2'. See the migration guide for more information: https://platform.openai.com/docs/assistants/migration.", 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_beta'}}